# Workshop: Introducción a Generative AI en Oracle y Creación de Agentes con LangChain

Bienvenidos al workshop. En esta sesión vamos a explorar cómo usar los **servicios de IA generativa de Oracle** para resolver problemas reales y luego **crear un agente inteligente** usando **LangChain** que pueda interactuar con estos servicios.

## Objetivos de la sesión
- Conocer la oferta de **Oracle Cloud Infrastructure (OCI Generative AI)** y cómo integrarla desde Python.
- Ejecutar peticiones a modelos de lenguaje para **generar texto** de manera controlada.
- Construir un **agente con LangChain** que use herramientas (como SQL o RAG) para responder preguntas de forma autónoma.
- Aprender buenas prácticas para **orquestar flujos de trabajo** y extender capacidades de los modelos.

## Requisitos previos
- Conocimientos básicos de Python 🐍
- Tener acceso a una cuenta de **Oracle Cloud** con permisos para usar **OCI Generative AI**
- Familiaridad básica con entornos virtuales y Jupyter Notebooks.

> 💡 **Tip:** este notebook está diseñado para ser práctico y paso a paso. Podrás copiar, ejecutar y modificar el código para experimentar con los conceptos que vamos a explicar.

¡Vamos a empezar!

## A continuación... 

📰 Recopilaremos noticias sobre el Fútbol argentino 

🤖 Consumiremos un modelo de lenguaje alojado en Oracle Cloud 

🔍 Construiremos un agente con langchain que es capaz de responder a preguntas relacionadas con el Fútbol argentino

## Instalación

In [1]:
!pip install tavily-python
!pip install -U langchain-community langchain-core oci

In [4]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient
import oci
import json
from oci.auth.signers import get_resource_principals_signer
from oci.config import from_file
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOCIGenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from IPython.display import Markdown, display
import json


Nuestro agente necesita acceso a la web para acceder a las últimas noticias del tema que le hemos indicado, para esto, es necesario configurar una herramienta que le permita a nuestro agente navegar por portales de noticias, por eso usaremos [Tavily](https://www.tavily.com/).

## 🪪 Registro en Tavily (paso a paso)

1) Abre **https://app.tavily.com/home** y haz clic en **Sign Up**. Verifica tu correo electrónico para activar la cuenta. ![image1](./images/tavily_signup.png)
2) Inicia sesión: en la **página principal** verás tu **API Key**. Haz **Copy** para copiarla. ![image2](./images/api_key.png)


In [5]:
# Pega la API Key de Tavily aquí
TAVILY_API_KEY = "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"

In [ ]:
assert TAVILY_API_KEY != "pega_aqui_tu_api_key", "Por favor, pega tu API Key de Tavily en la variable TAVILY_API_KEY"

In [8]:
ARG_FUTBOL_DOMAINS = [
    "ole.com.ar", "tycsports.com", "espn.com.ar", "lanacion.com.ar",
    "clarin.com", "infobae.com", "afa.com.ar", "promiedos.com.ar"
]
@tool
def get_futbol_argentino_comprehensive_news(pregunta:str=" Busca info de fútbol argentino (fixtures, TV, entradas, tabla, árbitros, etc.") -> str:
    """Devuelve info/noticias de Fiestas Patrias en Chile (fondas, ramadas, Parada Militar, feriados, normas)."""

    client = TavilyClient(TAVILY_API_KEY)
    response = client.search(
    query=pregunta,
    #include_domains=ARG_FUTBOL_DOMAINS,
    #topic="news",
    #days=45,
    #max_results=10
    )
    return response

## Configuración de la autenticación del SDK de OCI

Desde este notebook es necesario acceder a algunos servicios de Oracle, como el servicio de Generative AI, aunque ejecutes este notebook en cloud o de forma local, es necesario configurar las credenciales en la máquina que realiza el consumo del servicio. 

```
En los pasos anteriores fue necesario descargar un archivo terminado en .pem y copiar una configuración con el siguiente estilo
[DEFAULT]
user=ocid1.user.oc1..
fingerprint=95:e1:09
tenancy=ocid1.tenancy.oc1..
region= 
```

A continuación usaremos esos objetos.

In [9]:
# crea carpeta y permisos
!mkdir -p /home/datascience/.oci

In [10]:
# Ver tu HOME y listar (incluye ocultos)
!echo $HOME
!ls -la $HOME | head -n 30

/home/datascience
total 26214776
drwxr-xr-x. 13 datascience users         4096 Oct  6 19:46 .
drwxr-xr-x.  1 root        root          4096 Sep 30 00:48 ..
-rw-------.  1 nobody      nobody 26843545600 Oct  6 19:04 .__nb_swap__
drwxr-xr-x.  6 datascience users         4096 Sep  8 00:05 .cache
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:02 .config
drwxr-xr-x.  2 datascience users         4096 Oct  6 19:26 .ipynb_checkpoints
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:05 .ipython
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:02 .local
drwxr-x---.  2 datascience users         4096 Sep  8 00:02 .mnt
drwxr-xr-x.  2 datascience users         4096 Oct  6 19:12 .oci
drwxr-x---.  3 nobody      nobody        4096 Sep  8 00:02 .rclone
drwxr-xr-x.  2 datascience users         4096 Sep  8 00:02 .runtime-config-logs
-rw-r--r--.  1 datascience users        35405 Oct  6 19:46 agent_notebook.ipynb
-rw-r--r--.  1 datascience users       210801 Sep  8 07:11 demo-image.jp

In [11]:
!mkdir -p ~/.oci
!ls -la ~/.oci


total 16
drwxr-xr-x.  2 datascience users 4096 Oct  6 19:12 .
drwxr-xr-x. 13 datascience users 4096 Oct  6 19:46 ..
-rw-r--r--.  1 datascience users  308 Oct  6 19:12 config
-rw-------.  1 datascience users 1715 Sep  8 02:38 oci_api_key.pem


Ahora vamos a ubicar la llave privada que descargamos en los pasos anteriores al generar el API Key. El archivo tendrá un nombre similar a “tu_usuario-año-mes-diaTHH_MM_SS.XXX.pem”.

Este archivo debe renombrarse como “oci_api_key.pem” y cargarse en Data Science utilizando la opción “Upload Files”, o bien arrastrándolo directamente en el menú izquierdo del navegador.

Una vez que el archivo esté cargado, podremos proceder con la ejecución de la siguiente línea.

In [12]:
!mv ~/oci_api_key.pem ~/.oci/oci_api_key.pem
!chmod 600 ~/.oci/oci_api_key.pem
!ls -la ~/.oci

mv: cannot stat '/home/datascience/oci_api_key.pem': No such file or directory
total 16
drwxr-xr-x.  2 datascience users 4096 Oct  6 19:12 .
drwxr-xr-x. 13 datascience users 4096 Oct  6 19:46 ..
-rw-r--r--.  1 datascience users  308 Oct  6 19:12 config
-rw-------.  1 datascience users 1715 Sep  8 02:38 oci_api_key.pem


A continuación, crearemos el archivo de configuración en la ruta ~/.oci/config, vamos a copiar los valores de la configuración mostrada en pantalla y a reemplazarlos en la siguiente línea.

Reemplazaremos _ocid1.user.oc1.._ por el ocid del usuario mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
🚨 No reemplazaremos _key_file_ por ninguna ruta si estamos ejecutando este notebook en DataScience. Si queremos ejecutar este notebook de forma local, podemos reemplazar la ruta por ~/.oci/nombre_de_la_key.pem 

In [ ]:
%%bash
cat > ~/.oci/config <<'CFG'
[DEFAULT]
user=ocid1.user.oc1..
fingerprint=95:e1:09
tenancy=ocid1.tenancy.oc1..
region=
key_file=/home/datascience/.oci/oci_api_key.pem
CFG

echo "Config creado en ~/.oci/config"
cat ~/.oci/config | sed 's/fingerprint=.*/fingerprint=<oculto>/'

In [13]:
%%bash
cat > ~/.oci/config <<'CFG'
[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=f6:ec:66:50:04:b4:be:ca:55:c6:21:03:16:73:67:25
key_file=/home/datascience/.oci/oci_api_key.pem
CFG

echo "Config creado en ~/.oci/config"
cat ~/.oci/config | sed 's/fingerprint=.*/fingerprint=<oculto>/'

Config creado en ~/.oci/config
[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=<oculto>
key_file=/home/datascience/.oci/oci_api_key.pem


In [14]:
# Quita posibles finales de línea de Windows (CRLF)
!sed -i 's/\r$//' ~/.oci/config

# mostrar
!sed -n '1,200p' ~/.oci/config

[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=f6:ec:66:50:04:b4:be:ca:55:c6:21:03:16:73:67:25
key_file=/home/datascience/.oci/oci_api_key.pem


In [15]:
config = from_file()

In [17]:
# Descomenta únicamente la línea que corresponda a tu región
#REGION = "sa-saopaulo-1"
REGION = "us-chicago-1"
#REGION = "uk-london-1"
#REGION = "eu-frankfurt-1"
#REGION = "ap-osaka-1"

In [18]:
assert REGION in ["sa-saopaulo-1", "us-chicago-1", "uk-london-1", "eu-frankfurt-1", "ap-osaka-1"], "Por favor, descomenta la línea que corresponda a tu región"

In [19]:
# Aquí debes pegar el OCID de tu compartimento
# Encuentra el OCID de tu compartimento en la consola de Oracle Cloud, en la sección de Compartments https://cloud.oracle.com/identity/compartments
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaawy5gpq67r3rxc7yf6yn7oesvvgr56dqg7o5y3f5ndzt2mddtj46a"

In [20]:
SERVICE_ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"
MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya6dvgvvj3ovy4lerdl6fvx525x3yweacnrgn4ryfwwcoq"

In [23]:
if MODEL_ID is None:
    from oci.generative_ai import GenerativeAiClient
    genai = GenerativeAiClient()
    models = genai.list_models(
        compartment_id=COMPARTMENT_ID,
        capability=["CHAT"],
        lifecycle_state="ACTIVE"
    ).data.items
    assert models, "No hay modelos CHAT visibles en el compartimento. Revisa permisos/compartimento."
    MODEL_ID = models[0].id
    print("Usando modelo:", MODEL_ID)

# === Cliente de inferencia ===
inf = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config,
    service_endpoint=SERVICE_ENDPOINT
)

# === Prompt del usuario ===
user_input = "Qué partidos se juegan esta semana y dónde verlos??"

# --- Construcción del request ---
content = oci.generative_ai_inference.models.TextContent(text=user_input)
message = oci.generative_ai_inference.models.Message(role="USER", content=[content])

chat_request = oci.generative_ai_inference.models.GenericChatRequest(
    api_format=oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC,
    messages=[message],
    max_tokens=600,
    temperature=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    top_p=0.75,
)

chat_detail = oci.generative_ai_inference.models.ChatDetails(
    serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)

# === Llamada ===
resp = inf.chat(chat_detail)

# === Resultado ===
choices = resp.data.chat_response.choices
response_text = choices[0].message.content[0].text if choices else "No se generó respuesta."
print(json.dumps({"response": response_text}, indent=2, ensure_ascii=False))

{
  "response": "No mencionas una liga o deporte específico, así que asumiré que te refieres al fútbol, que es el deporte más popular en muchos países. Como no sé tu ubicación ni la semana exacta a la que te refieres, te daré una respuesta general basada en las principales ligas y competiciones que suelen jugarse durante una semana típica. Si quieres información más específica (por ejemplo, una liga concreta o una fecha exacta), por favor, detállalo.\n\n### Partidos destacados de esta semana (fútbol)\nVoy a basarme en las competiciones más relevantes que suelen tener partidos entre semana, como las ligas nacionales (Premier League, LaLiga, Serie A, etc.) y torneos internacionales (UEFA Champions League, Europa League, etc.). Los horarios y días exactos varían, pero te doy un resumen de lo que podrías esperar en una semana estándar de octubre o noviembre, que es cuando hay mucha actividad. Si es otra fecha, ajusta según el calendario.\n\n#### 1. UEFA Champions League y Europa League\n- 

## 🤖 Creación del Agente LangChain

In [24]:
# Configura tu endpoint y compartimento
ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"

llm = ChatOCIGenAI(
  model_id="meta.llama-4-maverick-17b-128e-instruct-fp8",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id=COMPARTMENT_ID,
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="DEFAULT"
)

tools = [get_futbol_argentino_comprehensive_news]

react_prompt_template = """Sos un asistente experto en **fútbol argentino**.
Respondé en español rioplatense y usá las herramientas cuando sumen valor.

Reglas:
- Tras usar una herramienta, **resumí con tus palabras** (no pegues el JSON).
- Priorizá info **reciente y confiable** (AFA, medios DEPORTIVOS locales).
- Incluí **fecha**, **hora**, **canal/stream**, **estadio**, **entradas** cuando aplique.
- Si hay versiones distintas entre medios, señalalo breve.

Herramientas disponibles:
{tools}

Formato EXACTO:
Question: la pregunta
Thought: qué vas a hacer
Action: una de [{tool_names}]
Action Input: el input
Observation: resultado de la acción
Thought: analizá y sintetizá
Final Answer: respuesta clara y útil

Empecemos.
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_prompt_template)
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    stream_runnable=False,
    handle_parsing_errors=True
)
preguntas_demo = [
    "¿Qué partidos se juegan esta semana en la Copa de la Liga y dónde verlos?",
    "Entradas para River vs. Boca, ¿precios y cómo comprar?",
    "Tabla y promedios actualizados, ¿quién pelea el descenso?",
    "¿Quién es el árbitro de Racing esta fecha?"
]
for p in preguntas_demo:
    print("\n\n=== PREGUNTA ===\n", p)
    out = agent_executor.invoke({"input": p})
    print("\n--- RESPUESTA ---\n", out["output"])



=== PREGUNTA ===
 ¿Qué partidos se juegan esta semana en la Copa de la Liga y dónde verlos?


> Entering new AgentExecutor chain...
Para responder a la pregunta, voy a buscar información sobre los partidos de la Copa de la Liga de esta semana, incluyendo detalles sobre dónde verlos.

Action: get_futbol_argentino_comprehensive_news
Action Input: "Copa de la Liga partidos esta semana TV"
{'query': 'Copa de la Liga partidos esta semana TV"\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.futbolred.com/parrilla-de-futbol', 'title': 'Toda la programación de Partidos de Fútbol por Tv', 'content': '!, Su correo ha sido verificado. Revise su bandeja de entrada y si no, en su carpeta de correo no deseado. | Santa Fe -  América | Liga Femenina | 4:00 p. | Orsomarso -  Inter Palmira | Liga Femenina | 5:00 p. | Deportivo Cali -  Unión Magdalena | Liga BetPlay | 6:30 p. | Millonarios -  Atlético Nacional | Liga Femenina | 3:00 p. | Botafogo -  Liga d

In [30]:
# === 1) Ejecutamos UNA PREGUNTA y guardamos 'respuesta' ===
pregunta = "¿Qué partidos se juegan esta semana en la Copa de la Liga y dónde verlos?"
respuesta = agent_executor.invoke({"input": pregunta})
print("\n--- RESPUESTA DEL AGENTE ---\n", respuesta["output"])

# === 2) Post-procesado / análisis en Markdown con la misma 'pregunta' y 'respuesta' ===
from IPython.display import Markdown, display
from langchain_core.messages import HumanMessage
import json

def _to_text(x):
    if isinstance(x, dict):
        return x.get("output") or json.dumps(x, ensure_ascii=False, indent=2)
    return str(x)

insumos = _to_text(respuesta)

analysis_prompt = f"""
Sos un analista especializado en **fútbol argentino**.
Usá solo los datos entregados más abajo para responder en **español rioplatense**, claro y útil.

**DATOS RECOLECTADOS** (pueden incluir JSON):  
{insumos}

---

### TAREAS
1. Respondé directamente a la pregunta: **"{pregunta}"**.
2. Extraé y organizá, si está disponible:
   - **torneo / fecha**
   - **partidos (local vs. visitante)**
   - **día y hora**
   - **estadio / ciudad**
   - **TV / streaming (canales, apps)**
   - **árbitro y VAR**
   - **entradas (venta, precios, links oficiales)**
   - **bajas/lesionados/suspendidos o convocados**
   - **tabla / posiciones / promedios**
   - **enlaces útiles**
3. Si hay diferencias entre medios, mencioná brevemente el conflicto y cuál parece más confiable.
4. Si algo **no figura en los datos**, marcá: *No disponible*.

---

### FORMATO (Markdown)

## Resumen
- 3–5 líneas con lo esencial (partidos clave, horarios, TV, nota destacada).

## Partidos y TV (si hay datos)
- **Partido** — Estadio — **Día/Hora** — **TV/Streaming** — Enlace

## Entradas y acceso (si hay datos)
- Punto breve con cómo comprar, precios y link oficial.

## Árbitros y Bajas (si hay datos)
- Árbitro/VAR; lesionados/suspendidos/convocados relevantes.

## Tabla y promedios (si hay datos)
- Nota corta o links a la tabla si aparece en los datos.

## Fuentes
- Lista de URLs citadas (solo si aparecen en los datos).

---

### REGLAS
- **No inventes** datos ni enlaces.
- Priorizá información **reciente y confiable** (AFA, TyC, Olé, ESPN, Promiedos).
"""

analysis_response = llm.invoke([HumanMessage(content=analysis_prompt)])
display(Markdown(analysis_response.content))



> Entering new AgentExecutor chain...
Para responder a la pregunta, voy a buscar información sobre los partidos de la Copa de la Liga de esta semana, incluyendo los horarios y dónde verlos.

Action: get_futbol_argentino_comprehensive_news
Action Input: "Copa de la Liga partidos esta semana horarios TV"
{'query': 'Copa de la Liga partidos esta semana horarios TV"\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.futbolred.com/parrilla-de-futbol', 'title': 'Toda la programación de Partidos de Fútbol por Tv', 'content': '!, Su correo ha sido verificado. Revise su bandeja de entrada y si no, en su carpeta de correo no deseado. | Santa Fe -  América | Liga Femenina | 4:00 p. | Orsomarso -  Inter Palmira | Liga Femenina | 5:00 p. | Deportivo Cali -  Unión Magdalena | Liga BetPlay | 6:30 p. | Millonarios -  Atlético Nacional | Liga Femenina | 3:00 p. | Botafogo -  Liga de Quito | Copa Libertadores | 5:00 p. | Independiente Medellín -  Deportivo 

## Resumen
- Esta semana en la Copa de la Liga se juegan dos partidos importantes: Vélez vs Lanús a las 21:30 y Defensa y Justicia vs Belgrano a las 16:45.
- Los partidos se pueden ver por ESPN.
- No hay información disponible sobre estadios, árbitros, bajas o tabla de posiciones.

## Partidos y TV
- **Vélez vs Lanús** — *No disponible* — **Día/Hora: esta semana a las 21:30** — **TV/Streaming: ESPN** — *No disponible*
- **Defensa y Justicia vs Belgrano** — *No disponible* — **Día/Hora: esta semana a las 16:45** — **TV/Streaming: ESPN** — *No disponible*

## Entradas y acceso
- *No disponible*

## Árbitros y Bajas
- *No disponible*

## Tabla y promedios
- *No disponible*

## Fuentes
- *No disponible*